In [2]:
try:
    import napari
    from napari.layers import Shapes
    from napari.utils.notifications import show_info
except:
    pass
import pandas as pd
import random
import tifffile as tiff

import dask.array as da
from dask.cache import Cache
import zarr
import os
import matplotlib.patches as mpatches
import numpy as np
import scipy.spatial.distance as sdistance
#from joblib import Parallel, delayed'
from magicgui import magicgui
import ast
from pathlib import Path

c:\Hyapp\Anaconda3-2020.11\envs\napari_tools\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Hyapp\Anaconda3-2020.11\envs\napari_tools\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Hyapp\Anaconda3-2020.11\envs\napari_tools\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [9]:
viewer = napari.Viewer()


c:\Hyapp\Anaconda3-2020.11\envs\napari_tools\lib\site-packages\napari_tools_menu\__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [10]:
@magicgui(call_button='Open image', layout='vertical')
def open_large_image(image_path=Path(), contrast_limit_txt=Path()):
    #ab = pd.read_csv("D://2022-06-20_Alfredos_project//template//cycle_1to7_renaming.txt")
    #ab = list(ab["ABS"])
    cl_txt = str(contrast_limit_txt)
    
    if cl_txt =='.':
        image = tiff.TiffFile(image_path, is_ome=False) #is_ome=False
        z = zarr.open(image.aszarr(), mode='r') # convert image to Zarr array
        # Identify the number of pyramids
        n_levels = len(image.series[0].levels) # pyramid

        # If and if not pyramids are available
        if n_levels > 1:
            pyramid = [da.from_zarr(z[i]) for i in range(n_levels)]
            multiscale = True
        else:
            pyramid = da.from_zarr(z)
            multiscale = False  

        viewer.add_image(
        pyramid, multiscale=multiscale, channel_axis=0,visible=False,)#name=ab

    else:
        cl = open(cl_txt, 'r')
        c = cl.read()
        contrast_limit = ast.literal_eval(c)
        
        image = tiff.TiffFile(image_path, is_ome=False) #is_ome=False
        z = zarr.open(image.aszarr(), mode='r') # convert image to Zarr array
        # Identify the number of pyramids
        n_levels = len(image.series[0].levels) # pyramid

        # If and if not pyramids are available
        if n_levels > 1:
            pyramid = [da.from_zarr(z[i]) for i in range(n_levels)]
            multiscale = True
        else:
            pyramid = da.from_zarr(z)
            multiscale = False  

        viewer.add_image(
        pyramid, multiscale=multiscale, channel_axis=0,visible=False,contrast_limits=contrast_limit,name=ab)
        cl.close()
        
viewer.window.add_dock_widget(open_large_image)


hii
hii
hii


In [23]:
import tifffile
screenshot = viewer.screenshot()
tifffile.imsave("E://2022-06-20_Alfredos_project//FAHNSCC11snapshots//masksandnuclei_CD45white_foxp3magenta_panckorange_ki67cyan_.jpg",screenshot)

C:\Users\junqmada\AppData\Local\Temp\ipykernel_23336\2100558209.py:3: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave("E://2022-06-20_Alfredos_project//FAHNSCC11snapshots//masksandnuclei_CD45white_foxp3magenta_panckorange_ki67cyan_.jpg",screenshot)


In [11]:
@magicgui(call_button='Save contrast limits', layout='vertical', output_file={"mode": "w"})
def save_contrast_limits(output_file=Path()):
    contrast_limit = list()
    ab = pd.read_csv("D://2022-06-20_Alfredos_project//template//cycle_1to9_renaming.txt")
    ab = list(ab["ABS"])
    for antibody in ab:
        contrast_limit.append(viewer.layers[antibody].contrast_limits)

    with open(output_file, "w") as output:
        output.write(str(contrast_limit))

viewer.window.add_dock_widget(save_contrast_limits)

In [45]:
@magicgui(call_button='Save removable cells to csv', filepath={"mode": "w", "mode": "d"})  
def save_list(filepath: Path):
    dict = {'cellid': removable_cells}
    df = pd.DataFrame(dict)
    df = df.astype(int)
    df.to_csv(str(filepath) + viewer.active_layer.name + '_cellids.csv', index=False)

viewer.window.add_dock_widget(save_list)

FIGURES FOR PUBLICATION

In [17]:
def create_box(data):
    """Create integer box

    Parameters
    ----------
    data : (N, 2) array
        Points around which the box is created.

    Returns
    -------
    box : (2, 2) array
        Integrer extrema of the box.
    """
    min_val = data.min(axis=0)
    max_val = data.max(axis=0)
    tl = np.array([min_val[0], min_val[1]])
    br = np.array([max_val[0], max_val[1]])
    box = np.round(np.array([tl, br])).astype(int)
    return box


def crop(image, rectangle):
    """Create integer box

    Parameters
    ----------
    image : (N, M) array
        2D image.
    rectangle : (4, 2) array or (2, 2) array
        Rectangle for cropping.

    Returns
    -------
    cropped_image : (U, V) array
        Cropped 2D image.
    """
    min_val, max_val = create_box(rectangle)
    return image[min_val[0]: max_val[0], min_val[1]: max_val[1]]


#this would get the layers and then everything is merged to an RGB file
def blended_img(viewer):
    import napari
    import numpy as np
    
    blended = np.zeros(viewer.layers[0].data.shape + (4,))
    for layer in viewer.layers:
        if layer.visible and not isinstance(layer, napari.layers.Shapes):
            # normalize data by clims
            normalized_data = (layer.data - layer.contrast_limits[0]) / (
            layer.contrast_limits[1] - layer.contrast_limits[0]
        )
            colormapped_data = layer.colormap.map(normalized_data.flatten())
            colormapped_data = colormapped_data.reshape(normalized_data.shape + (4,))

            blended = blended + colormapped_data
        
        blended[..., 3] = 1 # set alpha channel to 1

    return blended

In [119]:
def blended_img(viewer):
    blended = None
    layer_rectangle = viewer.layers["Shapes"]
    for i in range(len(layer_rectangle.data)):
        # iterate over layers to create new transformed layer only with those coordinates
        for layer in viewer.layers:
            if layer.visible and not isinstance(layer, napari.layers.Shapes):
                layer0 = np.array(layer_trial.data[0][int(layer_rectangle.data[i][0][0]):int(layer_rectangle.data[i][3][0]), int(layer_rectangle.data[0][0][1]):int(layer_rectangle.data[i][2][1])])
                viewer.add_image(layer0, name="transformed + str(i)")
        # iterate over the transformed layers to then create a composite
        for layer in viewer.layers:
           if "transformed" in layer.name:
            normalized_data = (layer.data - layer.contrast_limits[0]) / (layer.contrast_limits[1] - layer.contrast_limits[0])
            colormapped_data = layer.colormap.map(normalized_data.flatten())
            colormapped_data = colormapped_data.reshape(normalized_data.shape + (4,))

            blended = blended + colormapped_data
        
        blended[..., 3] = 1 # Set alpha channel to 1 outside of mask


    return blended 

In [116]:
for layer in viewer.layers:
    if "Image" in layer.name:
        print("y")

y
y
y
y
y
y
y
y
y
y


In [ ]:
layer_image = viewer.add_image(blended_img(viewer), name='blended')

In [55]:
layer_rectangle = viewer.layers["Shapes"]
print(layer_rectangle.data)
rect_coords = layer_rectangle.data[0]
x_min, y_min = rect_coords.min(axis=0)
x_max, y_max = rect_coords.max(axis=0)
mask = np.zeros(viewer.layers[0].data.shape[:2], dtype=bool)

        

[array([[ 9725.39911223, 19002.28186953],
       [ 9725.39911223, 19914.15873861],
       [11289.67569253, 19914.15873861],
       [11289.67569253, 19002.28186953]]), array([[11097.30192921, 16091.03600208],
       [11097.30192921, 17069.6355689 ],
       [12243.94384589, 17069.6355689 ],
       [12243.94384589, 16091.03600208]])]


In [103]:
i=0
new_layer = layer_trial.data[0][int(layer_rectangle.data[i][0][0]):int(layer_rectangle.data[i][3][0]), int(layer_rectangle.data[0][0][1]):int(layer_rectangle.data[i][2][1])]

In [104]:
new_layer

dask.array<getitem, shape=(1564, 912), dtype=uint16, chunksize=(1024, 458), chunktype=numpy.ndarray>

In [57]:
layer_trial = viewer.layers["Image"]

In [102]:
layer_rectangle.data[i][2][1]

19914.158738613034

In [97]:
new_layer = layer_trial.data[0][int(9725.39911223):int(11289.67569253), int(19002.28186953):int(19914.15873861)]
new_layer

dask.array<getitem, shape=(1564, 912), dtype=uint16, chunksize=(1024, 458), chunktype=numpy.ndarray>

In [ ]:
viewer.add_image(new_layer, name='cropped')

In [56]:
layer_image = viewer.add_image(blended_img(viewer), name='blended')

TypeError: list indices must be integers or slices, not tuple

In [35]:
#MAKE SURE YOU HAVE ADJUSTED THE CONTRAST LIMITS AS YOU WISH BEFORE RUNNING


# Add image layer
layer_image = viewer.add_image(blended_img(viewer), name='blended')

# Add shape layer with Shapes tool
layer_rectangle = viewer.layers["Shapes"]

for i in range(0, len(layer_rectangle.data)):
    cropped_image = crop(layer_image.data, layer_rectangle.data[i])
    #viewer.add_image(cropped_image, name='cropped')
    imsave('E:\\screenshot_{}.png'.format(i), cropped_image)


TypeError: slice indices must be integers or None or have an __index__ method

MASK CUT


In [46]:
@magicgui(call_button='Cut Mask')
def cut_mask():
    mask_to_cut = viewer.layers.selection.active.data
    newmask = np.copy(mask_to_cut)
    shape = viewer.layers.selection.active.data.shape
    selected_area = viewer.layers['Shapes'].to_labels(labels_shape=shape)
    global removable_cells
    removable_cells =[]

    for i in range(0,mask_to_cut.shape[0]):
        for j in range(0,mask_to_cut.shape[1]):
            cell = mask_to_cut[i,j]
            if selected_area[i,j] > 0 and cell not in removable_cells and cell > 0:
                removable_cells.append(cell)
                print(cell)

    return(removable_cells)

@magicgui(call_button='Save removable cells to csv', filepath={"mode": "w", "mode": "d"})  
def save_list(filepath: Path):
    dict = {'cellid': removable_cells}
    df = pd.DataFrame(dict)
    df = df.astype(int)
    df.to_csv(str(filepath) + '/' + viewer.layers.selection.active.name + '_selected_cellids.csv', index=False)

viewer.window.add_dock_widget(cut_mask)
viewer.window.add_dock_widget(save_list)

57341
39636
65090
54809
61806
38113
48446
55938
26601
34812
49575
60557
15345
52431
24650
25181
18304
56486
42441
22854
47453
20186
23983
28978
41671
40919
63072
6620
11496
40029
61087
14369
13463
43177
44443
9836
58128
12761
56109
45657
51388
40867
51901
62011
35188
57683
60129
17260
10760
40662
48275
25608
51046
53509
26258
34504
19159
51935
43416
41757
9289
15259
34076
50156
34333
19142
16696
40713
19775
2823
44819
39704
26703
24325
36796
26241
44631
32981
25865
50071
41415
10914
8861
39208
49404
64782
61601
3404
35821
57906
16628
7270
60728
53834
55288
45076
51320
52141
20100
56725
32964
18697
5286
7082
42835
9255
8091
16747
16918
58778
49831
36916
25095
3284
2908
59308
20562
35393
9973
33375
9614
60762
60215
16884
49267
5166
24497
28499
18732
45709
21366
15516
14541
50755
31561
16559
48052
59975
51473
38319
50498
44733
15139
54518
12231
50413
30210
58829
53081
19450
28756
50344
50139
61874
44768
56674
42749
36984
57529
12334
60318
47248
43451
61122
53064
42800
55391
57016
52722
95